In [1]:
interp.load.module(ammonite.ops.Path("/Users/cshao/DeepLearning.scala/jupyter-script.sc"))

In [2]:
import java.io.FileWriter
import java.util.concurrent.atomic.AtomicInteger
import scala.concurrent.duration.{SECONDS, FiniteDuration}
import scala.Long
import scala.util.{Try, Success, Failure}
import com.typesafe.config.{Config, ConfigFactory}
import akka.actor.{ Actor, ActorLogging, ActorRef, ActorSystem, Props }
import akka.event.Logging
import akka.pattern.ask
import akka.util.Timeout
import akka.util.Timeout._
import scalaz.{@@}
import scalaz.{Functor, Applicative, Monad, Tag}
import scalaz.syntax.all._
import com.thoughtworks.tryt.covariant._
import com.thoughtworks.future._
import com.thoughtworks.continuation._
import com.thoughtworks.raii.covariant._
import com.thoughtworks.raii.asynchronous._
import com.thoughtworks.feature.Factory
// import com.thoughtworks.deeplearning.plugins._
import com.thoughtworks.each.Monadic._

import java.io.FileWriter

import java.util.concurrent.atomic.AtomicInteger

import scala.concurrent.duration.{SECONDS, FiniteDuration}

import scala.Long

import scala.util.{Try, Success, Failure}

import com.typesafe.config.{Config, ConfigFactory}

import akka.actor.{ Actor, ActorLogging, ActorRef, ActorSystem, Props }

import akka.event.Logging

import akka.pattern.ask

import akka.util.Timeout

import akka.util.Timeout._

import scalaz.{@@}

import scalaz.{Functor, Applicative, Monad, Tag}

import scalaz.syntax.all._

import com.thoughtworks.tryt.covariant._

import com.thoughtworks.future._

import com.thoughtworks.continuation._

import com.thoughtworks.raii.covariant._

import com.thoughtworks.raii.asynchronous._

import com.thoughtworks.feature.Factory
// import com.thoughtworks.deeplearning.plugins._

import com.thoughtworks.each.Monadic._

In [4]:
private case class Dispatch[A](makeDo: Function0[Do[A]])
private case class Result[A](result: Try[A])

sealed trait Remote

val Remote = TagOf[Remote]
    
type RemoteDo[A] = Do[A] @@ Remote


class RemoteActor extends Actor with ActorLogging {
    
  import RemoteActor._
    
  val actorRef = counter.getAndIncrement
    
  appender(s"Entered RemoteActor No.$actorRef")

  def receive: Actor.Receive = {
      case Dispatch(makeDo) => {
          log.info(s"Received Dispatch message $makeDo")
          makeDo().run.onComplete { result =>
              sender ! Result(result)
          }
      }
  }
}

case object RemoteActor {
    val counter = new AtomicInteger()
}

class Remote protected (val actorSystem: ActorSystem)(implicit val timeout: Timeout) {
    
  appender("Inside Remote constructor")

  import Remote._
  import RemoteDo._
  import actorSystem.dispatcher

  var currentActor: ActorRef = {
      appender("Creating currentActor")
      val actor = actorSystem.actorOf(Props(new RemoteActor))
      appender("actorSystem.actorOf returned.")
      actor
  }
    
  def jump: Do[ActorRef] = {
    appender("Invoked jump")
    Do.now {
        val newActor = actorSystem.actorOf(Props(new RemoteActor))
        currentActor = newActor
        newActor
    }
  }
    
  def dispatch[A](actor: ActorRef, makeDo: => Do[A]): Do[A] = {
      appender("Inside remote.dispatch")
      val resultFuture = (actor ? Dispatch(makeDo _)).toThoughtworksFuture
      Do.garbageCollected(resultFuture).map {
          appender("Inside Do(resultFuture)")
          result => result.asInstanceOf[Result[A]].result.get
      }
      
  }
    
  def dispatch[A](makeDo: => Do[A]): Do[A] = {
      dispatch(currentActor, makeDo)
  }
    
  def jumpd[A](makeDo: => Do[A]): Do[A] = monadic[Do] {
      appender("Invoked jumpd")
      dispatch(jump.each, makeDo).each
  }
    
    def jumpd2: Do[Unit] = Do.safeAsync { (continue: Unit => Trampoline[Unit]) =>
        
    }
    
    jumpd2.map { unit =>
        ???
    }
    
  final class ImplicitsApi {
      implicit val remoteDoMonad: Monad[RemoteDo] = new Monad[RemoteDo] {
        override def point[A](a: => A): RemoteDo[A] = Tag(Do.delay(a))
        override def bind[A, B](fa: RemoteDo[A])(f: (A) => RemoteDo[B]) = Tag(monadic[Do] {
            val cont = Tag.unwrap(f(Tag.unwrap(fa).each))
            jumpd(cont).each
        })
    }
  }
    
  @transient lazy val implicits = new ImplicitsApi
    
}

case object Remote {

  def apply(makeActorSystem: Do[ActorSystem]): Do[Remote] = {
    makeActorSystem.flatMap { actorSystem =>
      {
        Remote(actorSystem)
      }
    }
  }

  def apply(makeActorSystem: => ActorSystem): Do[Remote] = {
      apply(Do.resource {
          val actorSystem = makeActorSystem
          
          Resource(actorSystem, UnitContinuation.delay {
              appender("Shutting down actor system")
              import actorSystem.dispatcher
              val Future(TryT(tryFinalizer)) = actorSystem.terminate.toThoughtworksFuture
              val finalizer = tryFinalizer.map {
                  case Success(t) =>
                      appender(t)
                      appender("The end of actorSystem.")
                  case Failure(e) =>
                      appender(s"Exception in finalizer $e")
              }
              finalizer
          }.join)
      })
    }
}

val config: Config = ConfigFactory.parseString("""
      akka {
        loglevel = "DEBUG"
        actor {
          provider = remote
          debug {
            receive = on
          }
        }
        remote {
          enabled-transports = ["akka.remote.netty.tcp"]
          netty.tcp {
            hostname = "127.0.0.1"
            port = 2552
          }
       }
      }
    """)

def appender[A](s: A): Unit = {
    synchronized {
        val f = new FileWriter("/tmp/log.txt", true)
        try {
            publish.md(s.toString)
            f.write(s"$s\n")
            f.flush()
        } finally {
            f.close()
        }
    }
}

def remoteSpec: Unit = {
    sys.props("sun.io.serialization.extendedDebugInfo") = "true"
    
    appender("======")

    implicit val timeout: Timeout = FiniteDuration(21474835, SECONDS)

    /*
    val m: Do[Double] = monadic[Do] {
        appender("Inside m monadic[Do] magic block")
        val remote = Remote(ActorSystem("RemoteSpecActorSystem", config)).each
        val actor0 = remote.currentActor
        remote.dispatch(actor0, {
            val x = Do.now(2.0).each
            val actor1 = remote.jump.each
            remote.dispatch(actor1, {
                val y = Do.now(3.0).each
                Do.now(x*y)
            })
        }).each
    }

    val m2: Do[Double] = monadic[Do] {
        appender("Inside m2 monadic[Do] magic block")
        val remote = Remote(ActorSystem("RemoteSpecActorSystem", config)).each
        val tp = remote.dispatch {
            Do.now(2.0)
        }.each
        val anotherActor = remote.jump.each
        val tp2 = remote.dispatch(anotherActor, {
            Do.now(3.0)
        }).each
        tp * tp2
    }
    
    val m3: Do[Double] = monadic[Do] {
        appender("Inside m3 monadic[Do] magic block")
        val remote = Remote(ActorSystem("RemoteSpecActorSystem", config)).each
        import remote.jumpd
        
        0.0
        
    }
    */
    

    val m4: Do[Double] = {
        import RemoteDo._
        
        Remote(ActorSystem("RemoteSpecActorSystem", config)).flatMap { remote => {
            import remote.implicits._
            
            Tag.unwrap(monadic[RemoteDo] {
                val remoteDo: RemoteDo[Double] = Remote(Do.now(2.0))
                // RemoteDo[Double]
                // Do[Double] @@ Remote
                val x = remoteDo.each
                val y = Tag[](Do.now(3.0)).each
                x * y
            })
        }}
    }


    // val r = m3.run.blockingAwait

    // appender(s"The result of remoteSpec: $r")
}

/*
def remoteSpec: Unit = {
    sys.props("sun.io.serialization.extendedDebugInfo") = "true"

    implicit val timeout: Timeout = FiniteDuration(21474835, SECONDS)

    val makeRemote = Remote(ActorSystem("RemoteSpecActorSystem", config))

    val r = makeRemote.flatMap {
        appender("Inside makeRemote.flatMap")
        remote => {
            import remote.actorSystem.dispatcher
            import remote.{jump, dispatch}
            (dispatch {
                val hp = Factory[Builtins].newInstance()
                import hp.implicits._
                hp.DoubleLayer(2.0.forward).forward
            }).flatMap {
                tp => {
                    jump.flatMap {
                        anotherActor => {
                            dispatch(anotherActor, {
                                val hp2 = Factory[Builtins].newInstance()
                                import hp2.implicits._
                                hp2.DoubleLayer(3.0.forward).forward
                            }).flatMap { tp2 => {
                                Do.now(tp.data * tp2.data)
                            } }
                        }
                    }
                }
            }
        }
    }.run.blockingAwait

    appender(s"The result of remoteSpec: $r")
}
*/

cmd4.sc:205: value each is not a member of scalaz.@@[com.thoughtworks.raii.asynchronous.Do[Double],T]
                val x = Tag(Do.now(2.0)).each
                                         ^cmd4.sc:206: value each is not a member of scalaz.@@[com.thoughtworks.raii.asynchronous.Do[Double],T]
                val y = Tag(Do.now(3.0)).each
                                         ^

: 

In [ ]:
remoteSpec